In [42]:
import pandas as pd

In [43]:
ped = pd.read_csv("köln-schildergasse (west)-20180430-20200131-hour.csv", sep = ";")
holi = pd.read_csv("Feiertage_2018.01.01_2020.01.31.csv", sep = ";")
print(ped.head(3))
print(holi.head(3))



                     location        time of measurement  weekday  \
0  Schildergasse (West), Köln  2018-05-01 00:00:00 +0200  Tuesday   
1  Schildergasse (West), Köln  2018-05-01 01:00:00 +0200  Tuesday   
2  Schildergasse (West), Köln  2018-05-01 02:00:00 +0200  Tuesday   

   pedestrians count  temperature in ºc    weather condition  incidents  
0                  0                8.0  partly-cloudy-night        NaN  
1                  0                7.0  partly-cloudy-night        NaN  
2                146                7.0  partly-cloudy-night        NaN  
         date        name
0  2018-01-01     Neujahr
1  2018-03-30  Karfreitag
2  2018-04-01      Ostern


In [44]:
#Splitting up the Time of measurement into date and time
ped[["date", "time"]] = ped["time of measurement"].str.split(" ", 1, expand=True)
#Drpo Time of measuremnt, because it is split now and drop location
ped = ped.drop(["time of measurement", "location"], axis=1)
#Drop incidents, if only nan
ped.dropna(how='all', axis=1, inplace=True)

# restructure by date
ped = ped[ ["date", "time"] + [ col for col in ped.columns if col != "date" and col!= "time"] ]
#change name
holi = holi.rename(columns={"name": "holiday"})
#merge on date
ped = pd.merge(ped, holi, how = "left")

#put hollyday as 1 and no holyday as 0
ped["holiday"] = ped["holiday"].fillna(0)
ped["holiday"] = ped["holiday"].where(ped["holiday"] == 0, 1)

# clear time from the minute variation
ped["time"] = ped["time"].str.slice_replace(8, 16)

# Get a better insight into the weather data and missing data
print(ped["weather condition"].unique())
print(ped.isnull().sum() / ped.shape[0])

#See where the data are missing
null_data = ped[ped.isnull().any(axis=1)]
print(null_data)

['partly-cloudy-night' 'partly-cloudy-day' 'clear-night' 'clear-day'
 'rain' 'cloudy' 'fog' nan 'wind']
date                 0.000000
time                 0.000000
weekday              0.000000
pedestrians count    0.000000
temperature in ºc    0.001755
weather condition    0.001755
holiday              0.000000
dtype: float64
            date      time    weekday  pedestrians count  temperature in ºc  \
1086  2018-06-15  06:00:00     Friday                195                NaN   
1128  2018-06-17  00:00:00     Sunday                386                NaN   
1199  2018-06-19  23:00:00    Tuesday                233                NaN   
1236  2018-06-21  12:00:00   Thursday               5895                NaN   
1260  2018-06-22  12:00:00     Friday               5553                NaN   
1266  2018-06-22  18:00:00     Friday               7363                NaN   
1291  2018-06-23  19:00:00   Saturday               5852                NaN   
1319  2018-06-24  23:00:00     Sunday  

C:\Users\Biebert\AppData\Local\Temp\ipykernel_2476\1063994339.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  ped[["date", "time"]] = ped["time of measurement"].str.split(" ", 1, expand=True)


In [45]:
#because the data for temperature and weather condition are both missing, this can be dropped, due to the huge sample size
ped.dropna(inplace=True)


In [46]:
ped

,date,time,weekday,pedestrians count,temperature in ºc,weather condition,holiday
0,2018-05-01,00:00:00,Tuesday,0,8.0,partly-cloudy-night,1
1,2018-05-01,01:00:00,Tuesday,0,7.0,partly-cloudy-night,1
2,2018-05-01,02:00:00,Tuesday,146,7.0,partly-cloudy-night,1
3,2018-05-01,03:00:00,Tuesday,125,7.0,partly-cloudy-night,1
4,2018-05-01,04:00:00,Tuesday,84,6.0,partly-cloudy-night,1
...,...,...,...,...,...,...,...
15380,2020-01-31,19:00:00,Friday,5422,11.0,partly-cloudy-night,0
15381,2020-01-31,20:00:00,Friday,2691,11.0,partly-cloudy-night,0
15382,2020-01-31,21:00:00,Friday,1329,10.0,partly-cloudy-night,0
15383,2020-01-31,22:00:00,Friday,915,11.0,cloudy,0
